In [20]:
import arcpy

# Access current ArcGIS Pro project and map
project = arcpy.mp.ArcGISProject("CURRENT")
map_ = project.activeMap

# Input layer name (update if different in the map)
input_layer_name = "CAMSCenterlines"
output_layer_name = "CAMSCenterlines_Dissolved"

# Check if the layer exists in the map
input_layer = None
for lyr in map_.listLayers():
    if lyr.name == input_layer_name:
        input_layer = lyr
        break

if not input_layer:
    raise ValueError(f"Layer '{input_layer_name}' not found in the current map.")

# Get layer path from the source
input_path = input_layer.dataSource

# Get the geodatabase or folder path
if input_path.endswith(".shp"):
    output_path = input_path.replace(".shp", "_Dissolved.shp")
else:
    workspace = arcpy.env.workspace = arcpy.Describe(input_path).path
    output_path = arcpy.Describe(workspace).catalogPath + f"\\{output_layer_name}"

# Perform dissolve by FullName
arcpy.management.Dissolve(
    in_features=input_path,
    out_feature_class=output_path,
    dissolve_field="FullName",
    statistics_fields="",
    multi_part="SINGLE_PART",
    unsplit_lines="DISSOLVE_LINES"
)

print(f"✅ Dissolved shapefile created at: {output_path}")


✅ Dissolved shapefile created at: C:\Users\JLin\Downloads\CAMSCenterlines (1)\CAMSCenterlines_Dissolved.shp


In [21]:
# Cell 2: Get First Row Street from Standalone Table
import arcpy
import difflib

# Configuration
table_name = "Section List"
dissolved_shapefile = "CAMSCenterlines_Dissolved"

# Get current project
project = arcpy.mp.ArcGISProject("CURRENT")
map_ = project.activeMap

# Find table
table_path = None
for table in map_.listTables():
    if table.name == table_name:
        table_path = arcpy.Describe(table).catalogPath
        break

if not table_path:
    raise ValueError(f"Table '{table_name}' not found")

# Find dissolved shapefile
dissolved_path = None
for lyr in map_.listLayers():
    if lyr.name == dissolved_shapefile:
        dissolved_path = arcpy.Describe(lyr).catalogPath
        break

if not dissolved_path:
    raise ValueError(f"Dissolved shapefile '{dissolved_shapefile}' not found")

# Get field names from table
field_names = [f.name for f in arcpy.ListFields(table_path)]
print(f"Available table fields: {field_names}")

# Find the name field
name_field_table = None
for field in field_names:
    if 'name' in field.lower():
        name_field_table = field
        break

if not name_field_table:
    raise ValueError("Could not find 'Name' field in table")

# Get first row street name
with arcpy.da.SearchCursor(table_path, [name_field_table]) as cursor:
    first_row = next(cursor)
    target_street = str(first_row[0]).strip()

print(f"First row street: '{target_street}'")

# Fuzzy matching function
def fuzzy_match(target, choices, threshold=0.7):
    target_norm = str(target).lower().replace('.', '').strip()
    best_match = None
    best_score = 0
    
    for choice in choices:
        if not choice:
            continue
        choice_norm = str(choice).lower().replace('.', '').strip()
        score = difflib.SequenceMatcher(None, target_norm, choice_norm).ratio()
        
        if score > best_score and score >= threshold:
            best_match = choice
            best_score = score
    
    return best_match, best_score

# Get all street names from dissolved shapefile
dissolved_streets = set()
with arcpy.da.SearchCursor(dissolved_path, ["FullName"]) as cursor:
    for row in cursor:
        if row[0]:
            dissolved_streets.add(str(row[0]).strip())

# Find best match
match, score = fuzzy_match(target_street, dissolved_streets)
if match:
    print(f"✅ Matched '{target_street}' to '{match}' (score: {score:.2f})")
    matched_street = match
else:
    print(f"⚠️  No match found for '{target_street}', using exact name")
    matched_street = target_street

print(f"Using street: '{matched_street}'")

Available table fields: ['OBJECTID', 'Sec_ID', 'Name', 'From_', 'To', 'Length', 'Width', 'Lanes', 'Layer_Name']
First row street: 'Abbey St.'
✅ Matched 'Abbey St.' to 'Abbey St' (score: 1.00)
Using street: 'Abbey St'


In [22]:
# Cell 3: Create FirstRow_Matched_Segments
import arcpy
import os

# Configuration
dissolved_shapefile = "CAMSCenterlines_Dissolved"
output_segments = "FirstRow_Matched_Segments"

# Get current project
project = arcpy.mp.ArcGISProject("CURRENT")
map_ = project.activeMap

# Find dissolved shapefile
dissolved_path = None
for lyr in map_.listLayers():
    if lyr.name == dissolved_shapefile:
        dissolved_path = arcpy.Describe(lyr).catalogPath
        break

if not dissolved_path:
    raise ValueError(f"Dissolved shapefile '{dissolved_shapefile}' not found")

# Setup workspace
workspace = os.path.dirname(dissolved_path)
output_path = os.path.join(workspace, f"{output_segments}.shp")

print(f"Creating segments for: '{matched_street}'")
print(f"Output will be saved to: {output_path}")

# Select the matched street from dissolved shapefile
field_delimited = arcpy.AddFieldDelimiters(dissolved_path, "FullName")
# Fix the string escaping issue
safe_street_name = matched_street.replace("'", "''")
where_clause = f"{field_delimited} = '{safe_street_name}'"

print(f"Selection criteria: {where_clause}")

# Create a layer for selection
dissolved_layer = "dissolved_temp_layer"
arcpy.management.MakeFeatureLayer(dissolved_path, dissolved_layer)

# Select features
result = arcpy.management.SelectLayerByAttribute(
    in_layer_or_view=dissolved_layer,
    selection_type="NEW_SELECTION",
    where_clause=where_clause
)

# Check selection count
selection_count = int(arcpy.management.GetCount(dissolved_layer)[0])
print(f"Selected {selection_count} feature(s)")

if selection_count == 0:
    print("❌ No features selected. Available streets:")
    with arcpy.da.SearchCursor(dissolved_path, ["FullName"]) as cursor:
        for i, row in enumerate(cursor):
            if i < 10:
                print(f"  - {row[0]}")
            elif i == 10:
                print("  - ... (and more)")
                break
    raise ValueError(f"No street found matching '{matched_street}'")

# Copy selected features to new shapefile
arcpy.management.CopyFeatures(dissolved_layer, output_path)

# Add to map
map_.addDataFromPath(output_path)

print(f"✅ Created and added to map: {output_segments}")
print(f"Extracted {selection_count} street line(s) for '{matched_street}'")

# Clean up
arcpy.management.Delete(dissolved_layer)

Creating segments for: 'Abbey St'
Output will be saved to: C:\Users\JLin\Downloads\CAMSCenterlines (1)\FirstRow_Matched_Segments.shp
Selection criteria: "FullName" = 'Abbey St'
Selected 2 feature(s)
✅ Created and added to map: FirstRow_Matched_Segments
Extracted 2 street line(s) for 'Abbey St'


<Result 'true'>

In [23]:
# Cell 4: Create FirstRow_Intersections
import arcpy
import os

# Configuration
matched_segments_layer = "FirstRow_Matched_Segments"
dissolved_shapefile = "CAMSCenterlines_Dissolved"
output_intersections = "FirstRow_Intersections"
tolerance = 0.1

# Get current project
project = arcpy.mp.ArcGISProject("CURRENT")
map_ = project.activeMap

# Find layers
matched_segments_path = None
dissolved_path = None

for lyr in map_.listLayers():
    if lyr.name == matched_segments_layer:
        matched_segments_path = arcpy.Describe(lyr).catalogPath
    elif lyr.name == dissolved_shapefile:
        dissolved_path = arcpy.Describe(lyr).catalogPath

if not matched_segments_path:
    raise ValueError(f"Layer '{matched_segments_layer}' not found")
if not dissolved_path:
    raise ValueError(f"Layer '{dissolved_shapefile}' not found")

# Setup workspace
workspace = os.path.dirname(matched_segments_path)
output_path = os.path.join(workspace, f"{output_intersections}.shp")

print(f"Finding intersections between:")
print(f"  - Selected segments: {matched_segments_path}")
print(f"  - All dissolved streets: {dissolved_path}")
print(f"Output: {output_path}")

# Run intersection analysis
arcpy.analysis.Intersect(
    in_features=[matched_segments_path, dissolved_path],
    out_feature_class=output_path,
    join_attributes="ALL",
    cluster_tolerance="",
    output_type="POINT"
)

# Add fields for intersection information
arcpy.management.AddField(output_path, "Street1", "TEXT", field_length=100)
arcpy.management.AddField(output_path, "Street2", "TEXT", field_length=100)
arcpy.management.AddField(output_path, "IntName", "TEXT", field_length=200)

# Get field names to find FullName fields
field_names = [f.name for f in arcpy.ListFields(output_path)]
fullname_fields = [f for f in field_names if "FullName" in f]

print(f"Found FullName fields: {fullname_fields}")

# Populate intersection fields and remove self-intersections
unique_intersections = []

if len(fullname_fields) >= 2:
    with arcpy.da.UpdateCursor(output_path, ["SHAPE@"] + fullname_fields + ["Street1", "Street2", "IntName"]) as cursor:
        for row in cursor:
            point_geom = row[0]
            street1 = str(row[1]) if row[1] else ""
            street2 = str(row[2]) if row[2] else ""
            
            # Skip self-intersections (same street)
            if street1.lower() != street2.lower() and street1 and street2:
                int_name = f"{street1} & {street2}"
                
                # Check for duplicates within tolerance
                is_duplicate = False
                for existing_point, existing_name in unique_intersections:
                    distance = point_geom.distanceTo(existing_point)
                    if distance <= tolerance:
                        # Check if it's the same intersection (same streets)
                        if set([street1.lower(), street2.lower()]) == set(existing_name.split(' & ')):
                            is_duplicate = True
                            break
                
                if not is_duplicate:
                    unique_intersections.append((point_geom, int_name))
                    cursor.updateRow([point_geom] + list(row[1:-3]) + [street1, street2, int_name])
                else:
                    cursor.deleteRow()
            else:
                cursor.deleteRow()

# Add to map
map_.addDataFromPath(output_path)

print(f"✅ Created intersection points: {output_intersections}")
print(f"Found {len(unique_intersections)} unique intersections")

# Show intersection details
print("\nIntersection points created:")
with arcpy.da.SearchCursor(output_path, ["Street1", "Street2", "IntName"]) as cursor:
    for i, row in enumerate(cursor, 1):
        print(f"  {i}. {row[2]}")

Finding intersections between:
  - Selected segments: C:\Users\JLin\Downloads\CAMSCenterlines (1)\FirstRow_Matched_Segments.shp
  - All dissolved streets: C:\Users\JLin\Downloads\CAMSCenterlines (1)\CAMSCenterlines_Dissolved.shp
Output: C:\Users\JLin\Downloads\CAMSCenterlines (1)\FirstRow_Intersections.shp
Found FullName fields: ['FullName', 'FullName_1']
✅ Created intersection points: FirstRow_Intersections
Found 5 unique intersections

Intersection points created:
  1. Abbey St & Central Ave
  2. Abbey St & Common St
  3. Abbey St & Albert St
  4. Abbey St & Common St
  5. Abbey St & S Stimson Ave


In [24]:
# Cell 5: Create Selected_Split_Segments
import arcpy
import os

# Configuration
matched_segments_layer = "FirstRow_Matched_Segments"
intersections_layer = "FirstRow_Intersections"
output_segments = "Selected_Split_Segments"
tolerance = 1.0

# Get current project
project = arcpy.mp.ArcGISProject("CURRENT")
map_ = project.activeMap

# Find layers
matched_segments_path = None
intersections_path = None

for lyr in map_.listLayers():
    if lyr.name == matched_segments_layer:
        matched_segments_path = arcpy.Describe(lyr).catalogPath
    elif lyr.name == intersections_layer:
        intersections_path = arcpy.Describe(lyr).catalogPath

if not matched_segments_path:
    raise ValueError(f"Layer '{matched_segments_layer}' not found")
if not intersections_path:
    raise ValueError(f"Layer '{intersections_layer}' not found")

# Setup workspace
workspace = os.path.dirname(matched_segments_path)
spatial_ref = arcpy.Describe(matched_segments_path).spatialReference
output_path = os.path.join(workspace, f"{output_segments}.shp")

print(f"Splitting segments from: {matched_segments_path}")
print(f"Using intersections from: {intersections_path}")
print(f"Output: {output_path}")

# Load intersection points
intersections = []
with arcpy.da.SearchCursor(intersections_path, ["SHAPE@", "Street1", "Street2", "IntName"]) as cursor:
    for row in cursor:
        point_geom, street1, street2, int_name = row
        if street1 and street2 and int_name:
            intersections.append({
                'geometry': point_geom,
                'point': point_geom.firstPoint,
                'int_name': int_name,
                'street1': street1,
                'street2': street2
            })

print(f"Loaded {len(intersections)} intersection points")

# Create output feature class
arcpy.management.CreateFeatureclass(
    out_path=workspace,
    out_name=os.path.basename(output_path).replace('.shp', ''),
    geometry_type="POLYLINE",
    spatial_reference=spatial_ref
)

# Add fields
arcpy.management.AddField(output_path, "StreetName", "TEXT", field_length=100)
arcpy.management.AddField(output_path, "FromInt", "TEXT", field_length=150)
arcpy.management.AddField(output_path, "ToInt", "TEXT", field_length=150)
arcpy.management.AddField(output_path, "SegmentLen", "DOUBLE")
arcpy.management.AddField(output_path, "SegmentID", "TEXT", field_length=50)

# Copy original fields
original_fields = [f.name for f in arcpy.ListFields(matched_segments_path) 
                   if f.type not in ['OID', 'Geometry'] and f.name.upper() not in ['SHAPE_LENGTH', 'SHAPE_AREA']]

for field_name in original_fields:
    field_obj = arcpy.ListFields(matched_segments_path, field_name)[0]
    try:
        arcpy.management.AddField(output_path, field_name, field_obj.type, field_length=field_obj.length)
    except:
        continue

def split_line_at_intersections(line_geom, intersections, tolerance):
    """Split a line at intersection points"""
    if not intersections:
        return [line_geom], [("START", "END")]
    
    # Get measures for intersection points on this line
    points_with_measure = []
    for intersection in intersections:
        try:
            measure = line_geom.measureOnLine(intersection['point'])
            # Only include points that are actually on the line (not just nearby)
            if 0 <= measure <= line_geom.length:
                points_with_measure.append((measure, intersection))
        except:
            continue
    
    # Sort by position along the line
    points_with_measure.sort(key=lambda x: x[0])
    
    # Remove duplicate measures (points too close together)
    filtered_points = []
    for measure, intersection in points_with_measure:
        if not filtered_points or abs(measure - filtered_points[-1][0]) > tolerance:
            filtered_points.append((measure, intersection))
    
    points_with_measure = filtered_points
    
    if not points_with_measure:
        return [line_geom], [("START", "END")]
    
    # Create segments
    segments = []
    segment_info = []
    
    # Build measures list: start, intersection points, end
    measures = [0.0]
    intersection_names = [None]
    
    for measure, intersection in points_with_measure:
        measures.append(measure)
        intersection_names.append(intersection['int_name'])
    
    measures.append(line_geom.length)
    intersection_names.append(None)
    
    # Create segments between consecutive measures
    for i in range(len(measures) - 1):
        start_measure = measures[i]
        end_measure = measures[i + 1]
        
        # Skip very short segments
        if end_measure - start_measure < tolerance:
            continue
        
        try:
            segment = line_geom.segmentAlongLine(start_measure, end_measure)
            if segment and segment.length > tolerance:
                segments.append(segment)
                
                from_int = intersection_names[i] if intersection_names[i] else "START"
                to_int = intersection_names[i + 1] if intersection_names[i + 1] else "END"
                
                segment_info.append((from_int, to_int))
        except Exception as e:
            print(f"    Error creating segment: {e}")
            continue
    
    # If no segments were created, return the original line
    if not segments:
        return [line_geom], [("START", "END")]
    
    return segments, segment_info

# Split the matched segments at intersections
created_segments = 0

with arcpy.da.SearchCursor(matched_segments_path, ["SHAPE@", "FullName"] + original_fields) as cursor:
    for row in cursor:
        line_geom = row[0]
        street_name = row[1] if row[1] else "Unknown"
        original_attributes = row[2:]
        
        print(f"\nSplitting street: {street_name}")
        
        # Split the line at intersection points
        segments, segment_info = split_line_at_intersections(line_geom, intersections, tolerance)
        print(f"  Created {len(segments)} segments")
        
        # Insert segments
        with arcpy.da.InsertCursor(output_path, 
                                   ["SHAPE@", "StreetName", "FromInt", "ToInt", "SegmentLen", "SegmentID"] + original_fields) as insert_cursor:
            
            for i, (segment_geom, (from_int, to_int)) in enumerate(zip(segments, segment_info)):
                segment_id = f"{street_name}_Seg_{i + 1}"
                
                insert_cursor.insertRow([
                    segment_geom,
                    street_name,
                    from_int,
                    to_int,
                    segment_geom.length,
                    segment_id
                ] + list(original_attributes))
                
                print(f"    Segment {i+1}: {from_int} → {to_int} ({segment_geom.length:.1f} units)")
        
        created_segments += len(segments)

# Add to map
map_.addDataFromPath(output_path)

print(f"\n✅ Created split segments: {output_segments}")
print(f"Total segments created: {created_segments}")

Splitting segments from: C:\Users\JLin\Downloads\CAMSCenterlines (1)\FirstRow_Matched_Segments.shp
Using intersections from: C:\Users\JLin\Downloads\CAMSCenterlines (1)\FirstRow_Intersections.shp
Output: C:\Users\JLin\Downloads\CAMSCenterlines (1)\Selected_Split_Segments.shp
Loaded 5 intersection points

Splitting street: Abbey St
  Created 1 segments
    Segment 1: Abbey St & Common St → Abbey St & Central Ave (817.8 units)

Splitting street: Abbey St
  Created 3 segments
    Segment 1: Abbey St & S Stimson Ave → Abbey St & Albert St (551.0 units)
    Segment 2: Abbey St & Albert St → Abbey St & Common St (669.1 units)
    Segment 3: Abbey St & Common St → Abbey St & Central Ave (2.6 units)

✅ Created split segments: Selected_Split_Segments
Total segments created: 4


In [25]:
# Summary Report
print("\n" + "="*60)
print("🎉 COMPLETE WORKFLOW FINISHED SUCCESSFULLY!")
print("="*60)
print("\n📋 LAYERS CREATED:")
print("1. ✅ CAMSCenterlines_Dissolved - All street centerlines dissolved by name")
print("2. ✅ FirstRow_Matched_Segments - Street segment from first table row")
print("3. ✅ FirstRow_Intersections - Intersection points where streets cross")
print("4. ✅ Selected_Split_Segments - Street split into segments at intersections")

print(f"\n📊 RESULTS:")
print(f"- Target street: '{matched_street}'")
print(f"- Intersection points found: {len(intersections)}")
print(f"- Final segments created: {created_segments}")

print(f"\n💡 USAGE:")
print("- Use 'Selected_Split_Segments' for your analysis")
print("- Each segment shows FromInt and ToInt intersection names")
print("- SegmentLen field contains the length of each segment")
print("- All original attributes are preserved")

print("\n🗂️  All layers have been added to your ArcGIS Pro map!")
print("="*60)


🎉 COMPLETE WORKFLOW FINISHED SUCCESSFULLY!

📋 LAYERS CREATED:
1. ✅ CAMSCenterlines_Dissolved - All street centerlines dissolved by name
2. ✅ FirstRow_Matched_Segments - Street segment from first table row
3. ✅ FirstRow_Intersections - Intersection points where streets cross
4. ✅ Selected_Split_Segments - Street split into segments at intersections

📊 RESULTS:
- Target street: 'Abbey St'
- Intersection points found: 5
- Final segments created: 4

💡 USAGE:
- Use 'Selected_Split_Segments' for your analysis
- Each segment shows FromInt and ToInt intersection names
- SegmentLen field contains the length of each segment
- All original attributes are preserved

🗂️  All layers have been added to your ArcGIS Pro map!


In [28]:
# Cell 6: Select Split Segment Matching First Table Row
import arcpy
import os

# Configuration
split_segments_layer = "Selected_Split_Segments"
table_name = "Section List"
output_final_segment = "Final_Matched_Segment"

# Get current project
project = arcpy.mp.ArcGISProject("CURRENT")
map_ = project.activeMap

# Find split segments layer
split_segments_path = None
for lyr in map_.listLayers():
    if lyr.name == split_segments_layer:
        split_segments_path = arcpy.Describe(lyr).catalogPath
        break

if not split_segments_path:
    raise ValueError(f"Layer '{split_segments_layer}' not found")

# Find table
table_path = None
for table in map_.listTables():
    if table.name == table_name:
        table_path = arcpy.Describe(table).catalogPath
        break

if not table_path:
    raise ValueError(f"Table '{table_name}' not found")

# Get first row data from table
field_names = [f.name for f in arcpy.ListFields(table_path)]
name_field = None
from_field = None
to_field = None

for field in field_names:
    field_lower = field.lower()
    if 'name' in field_lower and not name_field:
        name_field = field
    elif field_lower in ['from', 'from_'] or 'from' in field_lower:
        from_field = field
    elif field_lower in ['to', 'to_'] or field_lower.endswith('to'):
        to_field = field

# Get first row data
with arcpy.da.SearchCursor(table_path, [name_field, from_field, to_field]) as cursor:
    first_row = next(cursor)
    target_street, from_street, to_street = first_row
    target_street = str(target_street).strip()
    from_street = str(from_street).strip()
    to_street = str(to_street).strip()

print(f"Looking for segment: {target_street} from {from_street} to {to_street}")

# Function to normalize street names for better matching
def normalize_street_name(name):
    """Normalize street names by removing suffixes and punctuation"""
    if not name:
        return ""
    
    name = str(name).lower().replace('.', '').replace(',', '').strip()
    
    # Standardize common abbreviations
    replacements = {
        ' avenue': ' ave',
        ' street': ' st',
        ' drive': ' dr',
        ' road': ' rd',
        ' boulevard': ' blvd',
        ' lane': ' ln',
        ' court': ' ct'
    }
    
    for old, new in replacements.items():
        name = name.replace(old, new)
    
    return name

# Function to check if intersection contains street name (with normalization)
def intersection_contains_street(intersection_name, street_name):
    """Check if an intersection name contains a street name (normalized)"""
    if not intersection_name or not street_name:
        return False
    
    # Normalize both strings
    int_norm = normalize_street_name(intersection_name)
    street_norm = normalize_street_name(street_name)
    
    # Remove the suffix from street name to just match the base name
    street_parts = street_norm.split()
    if len(street_parts) >= 2:
        # Try matching with and without suffix
        street_base = ' '.join(street_parts[:-1])  # Everything except last word (suffix)
        return street_base in int_norm or street_norm in int_norm
    else:
        return street_norm in int_norm

# Find the matching segment
matching_segments = []

print("\nSearching through split segments...")
with arcpy.da.SearchCursor(split_segments_path, ["SHAPE@", "StreetName", "FromInt", "ToInt", "SegmentLen", "SegmentID"]) as cursor:
    for row in cursor:
        segment_geom, street_name, from_int, to_int, segment_len, segment_id = row
        
        print(f"  Checking: {segment_id}")
        print(f"    Street: {street_name}")
        print(f"    From: {from_int}")
        print(f"    To: {to_int}")
        
        # Check if the combined FromInt and ToInt contain both From_ and To streets
        combined_intersections = f"{from_int} {to_int}".lower()
        
        from_street_found = intersection_contains_street(combined_intersections, from_street)
        to_street_found = intersection_contains_street(combined_intersections, to_street)
        
        print(f"    Combined intersections: {combined_intersections}")
        print(f"    Looking for '{from_street}': {from_street_found}")
        print(f"    Looking for '{to_street}': {to_street_found}")
        
        if from_street_found and to_street_found:
            matching_segments.append({
                'geometry': segment_geom,
                'street_name': street_name,
                'from_int': from_int,
                'to_int': to_int,
                'segment_len': segment_len,
                'segment_id': segment_id
            })
            print(f"    ✅ MATCH FOUND! Both streets found in combined intersections")
        else:
            print(f"    ❌ No match")
            missing = []
            if not from_street_found:
                missing.append(f"'{from_street}'")
            if not to_street_found:
                missing.append(f"'{to_street}'")
            print(f"      Missing: {', '.join(missing)}")

print(f"\nFound {len(matching_segments)} matching segment(s)")

if not matching_segments:
    print("❌ No matching segments found!")
    print("\nAvailable segments:")
    with arcpy.da.SearchCursor(split_segments_path, ["SegmentID", "FromInt", "ToInt"]) as cursor:
        for row in cursor:
            print(f"  - {row[0]}: {row[1]} → {row[2]}")
    
    print(f"\nLooking for segment that connects:")
    print(f"  - An intersection containing '{from_street}'")
    print(f"  - An intersection containing '{to_street}'")
    
else:
    # Create output with the best matching segment
    workspace = os.path.dirname(split_segments_path)
    output_path = os.path.join(workspace, f"{output_final_segment}.shp")
    spatial_ref = arcpy.Describe(split_segments_path).spatialReference
    
    # Create output feature class
    arcpy.management.CreateFeatureclass(
        out_path=workspace,
        out_name=output_final_segment,
        geometry_type="POLYLINE",
        spatial_reference=spatial_ref
    )
    
    # Add fields
    arcpy.management.AddField(output_path, "StreetName", "TEXT", field_length=100)
    arcpy.management.AddField(output_path, "FromInt", "TEXT", field_length=150)
    arcpy.management.AddField(output_path, "ToInt", "TEXT", field_length=150)
    arcpy.management.AddField(output_path, "FromSt", "TEXT", field_length=100)
    arcpy.management.AddField(output_path, "ToSt", "TEXT", field_length=100)
    arcpy.management.AddField(output_path, "SegmentLen", "DOUBLE")
    arcpy.management.AddField(output_path, "SegmentID", "TEXT", field_length=50)
    arcpy.management.AddField(output_path, "Direction", "TEXT", field_length=20)
    arcpy.management.AddField(output_path, "TableMatch", "TEXT", field_length=5)
    
    # Insert the matching segment(s)
    with arcpy.da.InsertCursor(output_path, 
                               ["SHAPE@", "StreetName", "FromInt", "ToInt", "FromSt", "ToSt", 
                                "SegmentLen", "SegmentID", "Direction", "TableMatch"]) as cursor:
        
        for i, segment in enumerate(matching_segments):
            cursor.insertRow([
                segment['geometry'],
                segment['street_name'],
                segment['from_int'],
                segment['to_int'],
                from_street,
                to_street,
                segment['segment_len'],
                f"TableMatch_{i+1}",
                "matched",  # No longer tracking direction since we check combined
                "YES"
            ])
    
    # Add to map
    map_.addDataFromPath(output_path)
    
    print(f"✅ Created final segment: {output_final_segment}")
    print(f"Added to map with {len(matching_segments)} matching segment(s)")
    
    # Show details of matched segments
    print(f"\n📋 Matched Segment Details:")
    for i, segment in enumerate(matching_segments, 1):
        print(f"  {i}. {segment['segment_id']}")
        print(f"     Street: {segment['street_name']}")
        print(f"     From: {segment['from_int']}")
        print(f"     To: {segment['to_int']}")
        print(f"     Length: {segment['segment_len']:.1f} units")
        print(f"     ✅ Contains both '{from_street}' and '{to_street}' in intersections")
        print()

print("\n💡 Final segment represents the exact street section from your table!")
print("💡 FromSt/ToSt fields show the original table From/To values")
print("💡 FromInt/ToInt fields show the actual intersection names")
print("💡 The segment connects intersections that collectively contain both From and To streets")

Looking for segment: Abbey St. from Albert St. to Common Ave.

Searching through split segments...
  Checking: Abbey St_Seg_1
    Street: Abbey St
    From: Abbey St & Common St
    To: Abbey St & Central Ave
    Combined intersections: abbey st & common st abbey st & central ave
    Looking for 'Albert St.': False
    Looking for 'Common Ave.': True
    ❌ No match
      Missing: 'Albert St.'
  Checking: Abbey St_Seg_1
    Street: Abbey St
    From: Abbey St & S Stimson Ave
    To: Abbey St & Albert St
    Combined intersections: abbey st & s stimson ave abbey st & albert st
    Looking for 'Albert St.': True
    Looking for 'Common Ave.': False
    ❌ No match
      Missing: 'Common Ave.'
  Checking: Abbey St_Seg_2
    Street: Abbey St
    From: Abbey St & Albert St
    To: Abbey St & Common St
    Combined intersections: abbey st & albert st abbey st & common st
    Looking for 'Albert St.': True
    Looking for 'Common Ave.': True
    ✅ MATCH FOUND! Both streets found in combined int